In [3]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')  # Update the connection string with your MongoDB URI

# Access the database
db = client["Manuf"]
db
# Access the 'customer' collection


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Manuf')

### MANUF

### 1) Retrieve deliveries made to a specific customer for a particular product type:


In [2]:
db.delivery.find({ 'd_customerID': 78, 'd_productID': 104 })

In [2]:
collection = db['delivery']
pipeline = [
    {
        '$match': {
            'd_customerID': 78,
            'd_productID': 104
        }
    }
]

result = collection.aggregate(pipeline)

for document in result:
    print(document)

{'_id': ObjectId('656d482e5309ba5004c3c38c'), 'd_productID': 104, 'd_supplierID': 49, 'd_customerID': 78}


### 2) Retreive details of customer with 'customerContact': 7483472021 


In [36]:
db.customer.find({'customerContact': 7483472021 })

In [4]:
collection = db['customer']  

pipeline = [
    {"$match": {"customerContact": 7483472021}},
    {"$limit": 1}
]

result = list(collection.aggregate(pipeline))
print(result)

[{'_id': ObjectId('656d481d5309ba5004c3c328'), 'customerID': 2, 'customerName': 'Francisco Smith', 'customerAddress': '13266 Valerie Summit Apt. 055, North Karentown, NH 32458', 'customerContact': 7483472021}]


### 3) Calculate the count of deliveries made to each customer.


In [23]:
db.product.aggregate([
  {
    '$group': {
      '_id': "$p_manufacturingID",
      'totalProducts': { '$sum': 1 }
    }
  }
])

In [11]:
collection = db['delivery']  

pipeline = [
    {
        '$group': {
            '_id': "$d_customerID",
            'delivery_count': { '$sum': 1 }
        }
    }
]

result = list(collection.aggregate(pipeline))


### 4)  insert into  customer table

db.customer.insertOne({
    customerID: 1,
    customerName: 'John Doe',
    customerAddress: '123 Main St',
    customerContact: 1234567890
});

In [17]:
collection = db['customer']  

filter_query = {'customerID': 123}
update_query = {
    '$set': {
        'customerID': 123,
        'customerName': "John Doe",
        'customerAddress': "123 Main St",
        'customerContact': 1234567890
    }
}
options = {'upsert': True}

result = collection.update_one(filter_query, update_query, upsert=True)
print(result)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [19]:
collection = db['customer']  # Replace 'customer' with your collection name

# Define the pipeline for the update
pipeline = [
    {
        '$match': {
            'customerID': 123
        }
    },
    {
        '$set': {
            'customerName': "John Doe",
            'customerAddress': "123 Main St",
            'customerContact': 1234567890
        }
    },
    {
        '$merge': {
            'into': 'customer',
            'whenMatched': 'merge',
            'whenNotMatched': 'insert'
        }
    }
]

result = collection.aggregate(pipeline)
for doc in result:
    print(doc)
